In [1]:
2+2

4

In [2]:
# !pip list

In [3]:
# https://partner.steamgames.com/doc/store/getreviews

In [4]:
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [5]:
# Verify that the environment variable is loaded (optional for debugging)
# print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')}")

In [6]:
import requests
import json
import csv
import sys
import os
import time  # Import time for the epoch timestamp
import re  # Import re for regex


# Construct the path to the scripts directory
script_path = os.path.abspath('../reviews-assistant/scripts')

# Add the path to sys.path
if script_path not in sys.path:
    sys.path.append(script_path)

import minsearch


class SteamReviewFetcher:
    def __init__(self, appids_with_titles, filter="all", language="english", day_range=30, review_type="all", purchase_type="all"):
        """
        Initializes the SteamReviewFetcher with the required parameters.

        :param appids_with_titles: List of tuples containing Steam application IDs and their corresponding titles.
        :param filter: Type of review filter.
        :param language: Language of the reviews.
        :param day_range: Number of days to consider for reviews.
        :param review_type: Type of review (all or specific).
        :param purchase_type: Type of purchase.
        """
        self.base_url = "https://store.steampowered.com/appreviews/"
        self.appids_with_titles = appids_with_titles if isinstance(appids_with_titles, list) else [appids_with_titles]
        self.filter = filter
        self.language = language
        self.day_range = day_range
        self.review_type = review_type
        self.purchase_type = purchase_type
        self.data_dir = os.path.abspath('../reviews-assistant/data/reviews')

        # Ensure the data directory exists
        os.makedirs(self.data_dir, exist_ok=True)

    def _construct_url(self, appid):
        return f"{self.base_url}{appid}?json=1"

    def _fetch_reviews(self, appid, num_reviews):
        url = self._construct_url(appid)
        params = {
            "filter": self.filter,
            "language": self.language,
            "day_range": self.day_range,
            "review_type": self.review_type,
            "purchase_type": self.purchase_type,
            "num_per_page": num_reviews
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()

    def get_reviews(self, num_reviews=20, print_reviews=True):
        all_reviews = {}
        for appid, title in self.appids_with_titles:
            review_data = self._fetch_reviews(appid, num_reviews)
            reviews = review_data.get("reviews", [])
            all_reviews[appid] = (title, reviews)  # Store title along with reviews
            
            if print_reviews:
                self.print_first_last_reviews(appid, title, reviews)

        return all_reviews

    def print_first_last_reviews(self, appid, title, reviews):
        total_reviews = len(reviews)
        if total_reviews == 0:
            print(f"No reviews found for App ID {appid} ({title}).")
            return

        print(f"\nFirst 5 Reviews for App ID {appid} ({title}):")
        for review in reviews[:5]:
            self._print_review(review)

        print(f"\nLast 5 Reviews for App ID {appid} ({title}):")
        for review in reviews[-5:]:
            self._print_review(review)

    def _print_review(self, review):
        print(f"Author: {review['author']['steamid']}")
        print(f"Review: {review.get('review', 'No text')}")
        print(f"Rating: {'Positive' if review['voted_up'] else 'Negative'}")
        print(f"Timestamp: {review['timestamp_created']}")
        print("-" * 79)

    def _extract_columns_to_save(self, reviews, appid, title):
        extracted_reviews = []
        current_time = int(time.time())  # Get current epoch time
        for review in reviews:
            review_dict = {
                "appid": appid,  # Move appid to the first field
                "timestamp_query": current_time,  # Move timestamp_query to the second field
                "title": title,
            }
            for column in ["recommendationid", "author.steamid", "author.playtimeforever",
                           "author.playtime_last_two_weeks", "author.playtime_at_review",
                           "author.last_played", "language", "review", "voted_up", "votes_up", "timestamp_created", "timestamp_updated"]:
                if column.split('.')[0] == 'author':
                    nested_column = column.split('.')[1]
                    review_dict[column] = review['author'].get(nested_column)
                else:
                    review_dict[column] = review.get(column)
            extracted_reviews.append(review_dict)
        return extracted_reviews

    def save_reviews(self, all_reviews, filename_prefix, format):
        for appid, (title, reviews) in all_reviews.items():
            if not reviews:
                print(f"No reviews to save for App ID {appid} ({title}).")
                continue

            safe_title = self._sanitize_title(title)  # Sanitize title for filename
            lower_safe_title = safe_title.lower()  # Convert to lowercase for the filename
            if format == "csv":
                self._save_reviews_as_csv(appid, lower_safe_title, reviews, filename_prefix, title)
            elif format == "json":
                self._save_reviews_as_json(appid, lower_safe_title, reviews, filename_prefix, title)
            else:
                print("Invalid format. Please specify 'csv' or 'json'.")

    def _sanitize_title(self, title):
        """Remove special characters from the title for safe filename."""
        return re.sub(r'[<>:"/\\|?*]', '', title).replace("'", "").replace(" ", "_")

    def _save_reviews_as_csv(self, appid, lower_safe_title, reviews, filename_prefix, title):
        keys = ['appid', 'timestamp_query', 'title', 'steamid', 'review', 'voted_up', "votes_up", 'timestamp_created']  # Update the keys
        filename = os.path.join(self.data_dir, f"{filename_prefix}_{lower_safe_title}_{appid}_reviews.csv")
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(keys)  # Write the header
            for review in reviews:
                row = [
                    appid,  # App ID as the first field
                    int(time.time()),  # Current epoch timestamp as the second field
                    title,  # Keep the original title
                    review['author']['steamid'],
                    review.get('review', 'No text'),
                    'Positive' if review['voted_up'] else 'Negative',
                    review['timestamp_created'],
                ]
                writer.writerow(row)
        print(f"Reviews for App ID {appid} ({title}) saved to {filename}")

    def _save_reviews_as_json(self, appid, lower_safe_title, reviews, filename_prefix, title):
        filename = os.path.join(self.data_dir, f"reviews_{lower_safe_title}.json")
        extracted_reviews = self._extract_columns_to_save(reviews, appid, title)  # Use original title here
        with open(filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(extracted_reviews, jsonfile, indent=4)
        print(f"Reviews for App ID {appid} ({title}) saved to {filename}")


# Example usage
if __name__ == "__main__":
    appids_with_titles = [
        ("2322010", "God of War: Ragnarok"),
        ("1086940", "Baldur's Gate 3"),
        ("1680880", "Forspoken"),
        ("1496790", "Gotham Knights"),
        ("315210", "Suicide Squad: Kill the Justice League"),
        ("2443720", "Concord"),
        ("2208920", "Assassin's Creed Valhalla"),
        ("1817070", "Marvel’s Spider-Man Remastered"),
        ("1832040", "Flintlock: The Siege of Dawn"),
        ("2698940", "The Crew Motorfest"),
        ("2702430", "Usual June"),
        ("1545560", "Shadow Gambit: The Cursed Crew"),
        ("794540", "Neo Cab"),
        ("721180", "Dustborn"),
        ("1477940", "Unknown 9: Awakening"),
        ("2239550", "Watch Dogs: Legion"),
        ("447040", "Watch_Dogs 2"),
        ("243470", "Watch_Dogs"),
        ("582160", "Assassin's Creed Origin"),
        ("812140", "Assassin's Creed Odyssey"),
        ("552520", "Far Cry 5"),
        ("2369390", "Far Cry 6"),
        ("304390", "FOR HONOR"),
        ("2842040", "Star Wars Outlaws"),
        
    ]  # List of tuples containing app IDs and their titles
    review_fetcher = SteamReviewFetcher(appids_with_titles)

    num_reviews = 10000  # Specify the number of reviews to fetch
    print_reviews_flag = False  # Set to False to turn off printing reviews

    all_reviews = review_fetcher.get_reviews(num_reviews, print_reviews=print_reviews_flag)

    # Save reviews to file
    review_fetcher.save_reviews(all_reviews, "reviews", format="json")


Reviews for App ID 2322010 (God of War: Ragnarok) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_god_of_war_ragnarok.json
Reviews for App ID 1086940 (Baldur's Gate 3) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_baldurs_gate_3.json
Reviews for App ID 1680880 (Forspoken) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_forspoken.json
Reviews for App ID 1496790 (Gotham Knights) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_gotham_knights.json
Reviews for App ID 315210 (Suicide Squad: Kill the Justice League) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_suicide_squad_kill_the_justice_league.json
Reviews for App ID 2443720 (Concord) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_concord.json
Reviews for App ID 2208920 (Assassin's Creed Valhalla) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_assassins_creed_valhalla.json
Reviews for App ID 1817070 (Marvel’s Spider-Man Remastered) saved to /

# Ingestion

In [7]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list
# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561198420943538
Review: ---{ Graphics }---
✅ You forget what reality is
☐ Beautiful
☐ Good
☐ Decent
☐ Bad
☐ You will get eye cancer
☐ Get a pepper spray for your eye instead

---{ Gameplay }---
☐ Won’t ever touch any other game anymore
✅ Very good
☐ Good
☐ It's just gameplay
☐ Mehh
☐ Watch paint dry instead
☐ Tic Tac toe is better

---{ Audio }---
☐ Eargasm
✅ Very good
☐ Good
☐ Not too bad
☐ Bad
☐ I'm now deaf

---{ Audience }---
☐ Kids
✅Teens
✅ Adults
☐ Grandma

---{ PC Requirements }---
☐ Check if you can run paint
☐ Potato
☐ Decent
✅ Fast
☐ Rich boi
☐ Ask NASA if they have a spare computer
☐ Search the galaxy for dark matter fuel to run

---{ Difficulty }---
☐ Just press 'W'
☐ Easy
✅ Easy to learn / Hard to master
☐ Significant brain usage
☐ Difficult
☐ Dark Souls

---{ Grind }---
☐ Nothing to grind
☐ Only if u care about leaderboards/ranks
✅ Isn't necessary to progress
☐ Average grind level
☐ Too much grind
☐ You'll need a second life for grinding

---{ Story }---
☐ No

In [8]:
len(reviews)

1787

In [9]:
reviews[1]

{'appid': '812140',
 'timestamp_query': 1728201153,
 'title': "Assassin's Creed Odyssey",
 'recommendationid': '174039609',
 'author.steamid': '76561198085901084',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 101,
 'author.playtime_at_review': 245,
 'author.last_played': 1728159544,
 'language': 'english',
 'review': "I totally get why everyone hates on this game but I love it. When I used to have an Xbox I did played through this game twice and now that I'm replaying on PC I have fallen in love with it once more. This is better than Valhalla in my opinion and I like the world more than Origins, though I prefer the feel/gameplay of Origins. Still love this game and am planning on completing a playthrough on PC.",
 'voted_up': True,
 'votes_up': 12,
 'timestamp_created': 1725252188,
 'timestamp_updated': 1725252188}

In [10]:
reviews[-1]

{'appid': '2239550',
 'timestamp_query': 1728201153,
 'title': 'Watch Dogs: Legion',
 'recommendationid': '174862159',
 'author.steamid': '76561199371555316',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 1,
 'author.playtime_at_review': 109,
 'author.last_played': 1726993288,
 'language': 'english',
 'review': 'meh',
 'voted_up': True,
 'votes_up': 0,
 'timestamp_created': 1726233314,
 'timestamp_updated': 1726233314}

In [11]:
index = minsearch.Index(
    # text_fields=["author.steamid", "author.playtimeforever", "author.playtime_last_two_weeks", "author.playtime_at_review", "author.last_played", "language", "review", "timestamp_created", "timestamp_updated"],
    text_fields=["title", "language", "review"],
    keyword_fields=["appid", "recommendationid"]
)

In [12]:
index.fit(reviews)

# RAG flow

In [13]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [14]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [15]:
prompt_template = """
You're a conservative father of young children who is not aware of how the modern gaming industry works, who is not up to date with the titles released on a daily basis.
A father who needs to keep his child from not suitable games, including DEI games.
DEI ideology, which enforces Diversity, equity, and inclusion (DEI) hurts creativity, and uses corporate propaganda to improve sales.
Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
title: {title}
language: {language}
review: {review}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
prompt_template = """
You're a PC video game enthusiast who loves playing games on release day.
You're aware that it's now common for unfinished, unoptimized products to be released.
You know that developers often fail to deliver what was promised beforehand about the game.
You prefer to play games that simply work well and are delivered with proper quality—games that aren't released too early.
Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
title: {title}
language: {language}
review: {review}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [19]:
question = "Is Baldur's Gate 3 a game for kids?"
answer = rag(question)
print(answer)

Based on the context provided, Baldur's Gate 3 is not specifically categorized as a game for kids. The reviews highlight its complexity, depth, and the presence of mature themes typical in RPGs and Dungeons & Dragons mechanics. It is recommended for players who enjoy rich narratives, character development, and intricate gameplay, which may be more appealing to older audiences. Additionally, some reviews mention the game's potential challenges for those unfamiliar with D&D, suggesting that it is more suited for players who can grasp its intricacies rather than a young demographic.


In [20]:
question = "Is Baldur's Gate 3 a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Based on the provided reviews, Baldur's Gate 3 features a diverse cast of characters and offers players a variety of choices and playstyles, reflecting elements of diversity, equity, and inclusion (DEI). The game includes different character options and allows for unique adventures based on player decisions, contributing to an immersive experience that many players appreciate. However, the reviews do not explicitly discuss DEI principles, leaving some aspects open to interpretation. Overall, Baldur's Gate 3 seems to align with DEI concepts through its emphasis on diverse character representation and player agency.


In [21]:
question = "Is God of War Ragnarök a game for kids?"
answer = rag(question)
print(answer)

Based on the context provided, "God of War Ragnarök" is not a game specifically designed for kids. The game's themes and narrative are complex and often delve into darker, more mature subjects. The review mentions elements such as desperation, end-of-times settings, and character development that might not be suitable for a younger audience. Additionally, the action and combat, while engaging, are geared towards a more mature gaming demographic. Overall, while younger players may enjoy the gameplay, they might not fully grasp the deeper narrative and themes present in the game.


In [22]:
question = "Is God of War Ragnarök a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Based on the provided context, "God of War Ragnarök" does exhibit elements that align with DEI (diversity, equity, and inclusion) concepts. The developers collaborated with Sweet Baby Inc., a company known for promoting inclusive storytelling, which influenced the game's narrative and character decisions. However, some reviewers express that this "woke influence" sometimes feels forced, detracting from the overall experience and narrative coherence. 

Therefore, while God of War Ragnarök incorporates DEI elements, its execution and impact on gameplay and storytelling have received mixed feedback from players.


In [23]:
question = "Is Far Cry 6 a game for kids?"
answer = rag(question)
print(answer)

Far Cry 6 is not specifically a game for kids. While it retains the typical gameplay mechanics of the Far Cry series—which include shooting, looting, and exploration—it tends to have themes and content more suited for a mature audience. Reviews mention that the series has historically featured immersive stories with complex characters, and overall, Far Cry games generally are rated for ages 17 and older due to violence, strong language, and other mature themes.


In [24]:
question = "Is Far Cry 6 a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

The reviews provide a mixed view on whether Far Cry 6 incorporates DEI concepts, namely diversity, equity, and inclusion. While some reviews reference diversity positively, there isn't a comprehensive assessment or explicit mention of DEI principles as a focus within the game. Many reviews indicate that Far Cry 6 feels similar to previous titles and lacks depth in story and character engagement. Therefore, while diversity may be present, the overall emphasis on equity and inclusion is not clearly highlighted or explored in the context of the reviews provided.


In [25]:
question = "Is Watch Dogs: Legion a game for kids?"
answer = rag(question)
print(answer)

Based on the reviews provided, Watch Dogs: Legion is not specifically tailored for kids. The game is set in a near-future dystopian London and deals with themes such as authoritarianism and surveillance. Additionally, it features complex gameplay mechanics and a narrative that may not resonate with a younger audience. The content seems to be more suitable for adult players who appreciate open-world games with strategic elements and social commentary, rather than a game designed for children.


In [26]:
question = "Is Watch Dogs: Legion a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Based on the provided context, there is no explicit mention of Watch Dogs: Legion directly addressing DEI concepts such as diversity, equity, and inclusion. However, the game's unique "Play as Anyone" mechanic allows players to recruit and control various characters from different backgrounds, which suggests a degree of diversity in gameplay. The diversity of characters, from hackers to construction workers and grandmas, may imply some level of representation.

Despite this, the reviews predominantly focus on gameplay mechanics, storyline, and technical aspects rather than a clear indication of DEI principles being a core theme or focus within the game. Therefore, while there are elements that could align with diversity, the overall context does not provide sufficient evidence to conclude that the game fully embodies DEI concepts.


In [27]:
question = "Is Forspoken a game for kids?"
answer = rag(question)
print(answer)

Based on the provided context, there isn't any explicit information indicating that "Forspoken" is specifically a game for kids. The reviews highlight aspects such as unique gameplay, emotional storylines, and character development, but they do not suggest a target audience focused on children. Rather, the discussions seem to cater to a more general gaming audience, with considerations of gameplay complexity and mature themes. Thus, it is not accurate to classify "Forspoken" as a game solely for kids.


In [28]:
question = "Is Forspoken a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

The provided reviews for "Forspoken" do not specifically mention whether the game follows DEI (diversity, equity, and inclusion) concepts. However, it is noted that the game features a mostly female cast and includes a protagonist named Frey, which could indicate an element of diversity. The reviews primarily focus on aspects like gameplay mechanics, graphics, and personal enjoyment rather than explicitly addressing DEI themes. Therefore, based on the information in the context, it cannot be definitively stated that "Forspoken" embodies DEI concepts.


In [29]:
question = "Are there any technical issues when playing Baldur's Gate 3?"
answer = rag(question)
print(answer)

Based on the reviews provided, there are no reported technical issues when playing Baldur's Gate 3. The game is described as a polished AAA experience with high-quality voice acting, stunning visuals, and engaging gameplay. Players have expressed enjoyment and satisfaction with the game’s performance and depth. While one review mentions voice acting accents detracting from immersion, this seems to be a subjective opinion rather than a technical issue. Overall, players consider Baldur’s Gate 3 to be an outstanding and stable experience.


In [30]:
question = "Is the release version of Baldur's Gate 3 ready to be played?"
answer = rag(question)
print(answer)

Yes, the release version of Baldur's Gate 3 is indeed ready to be played. Reviews consistently praise it as an amazing and polished game, often referring to it as a masterpiece with a deep and engaging story, well-developed characters, and enjoyable gameplay mechanics. Players have invested significant hours into the game (some over 370 hours) and express excitement about replaying it due to its extensive content and choices. The game is also noted for its remarkable balance between narrative and emergent gameplay, making it a complete and satisfying experience. There's no mention of significant issues or that it's unfinished, suggesting that it meets the quality standards you prefer.


In [31]:
question = "Are there any technical issues when playing God of War Ragnarok?"
answer = rag(question)
print(answer)

Yes, there are several technical issues when playing God of War: Ragnarok on PC. Users have reported problems such as:

1. **VRAM Limitations**: The game displays an error message when trying to launch on systems with 4GB VRAM graphics cards, preventing them from playing. While there is a workaround using a mod, this limitation affects many players.

2. **Crashing**: Some players experience crashes every couple of hours of gameplay, which can be frustrating.

3. **Performance Issues**: Users have reported various performance problems, including freezing, stuttering, memory leaks, and input lag when using controllers.

4. **Audio Issues**: There are mentions of audio cutting in and out during gameplay.

5. **Frame Pacing Issues**: Some players with powerful PCs experience jerky and stuttery performance despite having good average frame rates, possibly related to frame pacing and VRR (Variable Refresh Rate) not functioning correctly.

Overall, many players suggest waiting for patches to 

In [32]:
question = "Is the release version of God of War Ragnarok ready to be played?"
answer = rag(question)
print(answer)

The release version of God of War: Ragnarok is not entirely ready to be played without issues. There are reports of users with 4GB VRAM graphics cards encountering a VRAM limitation that prevents the game from launching. While some players have found a workaround using a mod, such solutions indicate that the game may not fully support lower-end systems that meet the minimum requirements.

Additionally, players have experienced crashes every few hours, which is not typical for modern games, even on systems that exceed the minimum requirements. Although the game offers a great experience overall and is praised for its story, graphics, and gameplay, the presence of these technical issues suggests that the game is not optimized or fully ready for all players at launch. Therefore, while some can enjoy it without problems, others may face significant barriers.


In [33]:
question = "Are there any technical issues when playing Far Cry 6?"
answer = rag(question)
print(answer)

Yes, Far Cry 6 is plagued by technical issues. The game experiences frequent bugs and glitches, including problems like enemies that disappear, missions not triggering properly, and long load times. Additionally, even on higher-end systems, players have reported inconsistent performance with frame drops and stuttering during intense sequences. These issues detract from the overall experience and contribute to a feeling that the game is unfinished.


In [34]:
question = "Is the release version of Far Cry 6 ready to be played?"
answer = rag(question)
print(answer)

Based on the reviews provided, the release version of Far Cry 6 is not ready to be played without issues. The game is described as being heavily glitched and buggy, with one review suggesting it should have been treated as an alpha version rather than a finished product. Furthermore, warnings about performance issues, particularly when installed on a hard drive, indicate significant technical problems. Overall, it appears that while the gameplay may have its merits, the game suffers from substantial quality control issues that impact the playing experience.
